***
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This Runbook demonstrates usage of the K8S kubectl lego.
    Using this lego, we can query the K8S cluster and find out pod(s) stuck in Terminating state.
    </b>
</div>

<br>
<center><h2>K8S pod stuck in Terminating State</h2></center>

A Pod has been deleted but remains in `Terminating` Status.

This can happen for either of the reasons:

    1. Pod has a finalizer associated with it and that is not completing.
    2. The Pod is not responding to termination signals.
    

The output of `kubectl get pods [PODNAME] -n [NAMESPACE]` will show something like this:

```
NAME                     READY     STATUS             RESTARTS   AGE
nginx-7ef9efa7cd-qasd2   1/1       Terminating        0          1h
```


## Initial Steps
    1. Gather Information
    2. Check for finalizers
    2.1 Remove finalizer if present
    3. Get Node Information
    3.1 Check Node Status
    4. Force-delete the pod
    5. Check Resolution
    6. Further Steps


The Original Runbook Author is [Ian Miell](https://containersolutions.github.io/runbooks/posts/kubernetes/pod-stuck-in-terminating-state/) 
***

### 1. Gather Information

We will use `kubectl get pods -n {namespace} | grep Terminating `  to find out all the PODs that are stuck in 
the `Terminating` state for the given `namespace`. Please note, we will be using the unSkript's `kubectl lego` to accomplish this task. We then will store this list as `terminatingPods`.

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} | grep Terminating | cut -d' ' -f 1\\""
    }''')
task.configure(outputName="terminatingPods")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 2 Check for Finalizers

Here we will use unSkript `kubectl lego` to execute `kubectl get pod -n {namespace} {podName} -o yaml ` command.
We then observe the output of the command to find out if there are any Finalizers configured for the POD stuck in
`Terminating ` state. This Lego Demonstrates the `Start Condition ` feature of unSkript platform. We first check
step-1 found out Pods that were stuck in the `Terminating ` state and only then we proceed to further examine
if the Pod has any `Finalizers` configured in it. If Found, it will store in a variable called `finalizerOutput` 
which will be used the following cells.

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None
    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pod -n {namespace} {terminatingPods.strip()} -o yaml | grep -A 1 finalizers\\" "
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "terminatingPods is not ''",
    "condition_result": true
    }''')
task.configure(outputName="finalizerOutput")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 2.1 Remove Finalizers if present

Here we use the same versatile unskript `kubectl lego` to execute the `kubectl patch ... ` command 
to reset the Finalizer metadata to `null`,  Once done, the POD will exit out of the `terminating` state. 

This cell demonstrates the simplicity of the unSkript platform to perform complex operations like
patching the metadata of a k8s cluster. Once the `Finalizer` is removed, the output of the action is 
stored in the `removeFinalizerOutput` variable to be used further in the cells.

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "finalizerOutput is not ''",
    "condition_result": true
    }''')
task.configure(outputName="removeFinalizerOutput")

task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl patch pod {terminatingPods.strip()}\\" + \\" -p '{\\"metadata\\":{\\"finalizers\\":null}}'\\""
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 3 Get Node Information

Here we will use the unskript `kubectl lego` to execute `kubectl get pods` and find out the `NodeName` assigned
for this Pod. We use the `unSkript`  `Start Condition` feature to execute this command only when step `2.1` 
was successful. The output of the this lego will be saved in the variable `nodeName` which will be used 
later in the following cells.

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods {terminatingPods.strip()} -n {namespace} -o yaml | grep nodeName | tr -d \\" \\" | cut -d':' -f 2\\" "
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": false,
    "condition_cfg": "removeFinalizerOutput is ''",
    "condition_result": true
    }''')
task.configure(outputName="nodeName")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 3.1 Check Node Status

Here we check the status of the `NodeName` from step `3.0`. We will use the unSkript `kubectl lego` 
to perform `kubectl get pods -n {namespace} {nodeName}`. Needless to say, we use the `Start Condition` 
feature again. The output of the kubectl command will be stored in `nodeStatus` variable which will
be used later in the following cells.

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} {nodeName} | grep \\" Ready\\" | cut -d' ' -f 1 | tr -d ' '\\" "
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "nodeName is not ''",
    "condition_result": true
    }''')
task.configure(outputName="nodeStatus")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)


### 4 Force Delete the Pod

Here again we will use unSkript `kubectl lego` to perform `kubectl delete pod {podname} -n {namespace} --now`. 
This lego again uses the `Start Condition` to make sure we execute this command only when we determine
the output `terminatingPods` is not Empty. 

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "terminatingPods is not ''",
    "condition_result": true
    }''')
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl delete pod {terminatingPods.strip()} -n {namespace} --now\\""
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 5 Check Resolution

This Lego like all previous legos in this runbook uses unSkript `kubectl lego` one final time to
check the status of the Deletion of the pod in `Step 4`. We perform `kubectl get pods -n {namespace}`
one last time to list all the existing PODs. If all the steps above were successful, then the list
of pods should not display the `terminating pod`. 

In [87]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace}\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 6 Further Steps

If For some reason `Step 5` displayed the offending Pod in the list of pods, then here are some further
debugging steps recommended to resolve the issue.

***
If the POD still stuck in `Terminating` state then you can consider.

    1. Restarting kubelet
        
        If you can SSH to the node and restart the kublet process. You may need
        administrator privileges to do so. Before you do that, you may also want
        to check the kubelet logs for any issues.
        
    2. Check Whether finalizer's work needs to get done before termination
    
        This will vary depending on what the finalizer is doing. Please refer to 
        [Finalizers](https://kubernetes.io/docs/tasks/extend-kubernetes/custom-resources/custom-resource-definitions/#finalizers). Common cases for finalizers not completing relates to
        Volumes.
        
***

### Conclusion

In this Runbook, we demonstrated that using simple  `unSkript Legos` combined with the Platform features like
`Start Condition` we can accomplish a complex workflow such as patching a K8S cluster to resolve the issue of
POD stuck in the `Terminating` state. 

To experience the Full platform capability of `unSkript` please visit [www.unskript.com](https://www.unskript.com).